In [1]:
import cv2
import numpy as np
from tqdm import tqdm
import math
from glob import glob
import os

In [2]:
readdir = r'D:\data\widefield_eyecams\raw_data'
savedir = r'D:\data\widefield_eyecams\training_data'

In [3]:
input_list = glob(os.path.join(readdir, '*.avi'))

In [12]:
for video in input_list:
    # build the save path
    head, tail = os.path.split(video)
    split_name = tail.split('_')
    new_name = '_'.join(split_name[:3])+'Gammacor_'+'_'.join(split_name[3:])
    savepath = os.path.join(savedir, new_name)

    # write new video with gamma correction
    vid_read = cv2.VideoCapture(video)
    width = int(vid_read.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid_read.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out_vid = cv2.VideoWriter(savepath, fourcc, 60.0, (width, height))

    num_frames = int(vid_read.get(cv2.CAP_PROP_FRAME_COUNT))
#     num_save_frames = 3600
    # gamma = 0.4

    for step in tqdm(range(0,num_frames)):
        ret, frame = vid_read.read()
        if not ret:
            break
        # convert img to gray
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # compute gamma = log(mid*255)/log(mean)
        mid = 0.5
        mean = np.mean(gray)
        gamma = math.log(mid*255)/math.log(mean)

        # do gamma correction
        img_gamma1 = np.power(frame, gamma).clip(0,255).astype(np.uint8)

        out_vid.write(img_gamma1)
    out_vid.release()

100%|█████████████████████████████████████████████████████████████████████████▉| 19097/19098 [1:55:28<00:00,  2.85it/s]